In [2]:
# function to upload json file to S3 for staging
from s3 import upload_file
import requests as r
import json
from config import X_RAPIDAPI_KEY

In [5]:
url_teams = "https://api-football-v1.p.rapidapi.com/v3/teams"

# world cup season 2022 
params = {
    "league":"1",
    "season":"2022"
}

headers = {
    "X-RapidAPI-Key": X_RAPIDAPI_KEY,
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

data = r.request("GET", url_teams, headers=headers, params=params).json()

upload_file(data, 'worldcup-project-2022-bucket', 'worldcup_teams_2022.json')

True